# Семинар 4. Sequence Labeling (POS + NER)

## Введение: Что такое Token Classification?

В этом ноутбуке мы работаем с классом задач Sequence Labeling (разметка последовательности). В отличие от классификации всего текста (Text Classification), здесь мы принимаем решение для каждого токена отдельно.

Основные задачи:

1.  POS (Part-of-Speech tagging). Определение части речи.
      * *Пример:* `Мама (NOUN) мыла (VERB) раму (NOUN)`.
2.  NER (Named Entity Recognition). Извлечение именованных сущностей.
      * *Пример:* `Илон (B-PER) Маск (I-PER) купил (O) Twitter (B-ORG)`.

**Форматы разметки (BIO / BIOES)**

Чтобы модель понимала, где начинается и где заканчивается сущность (особенно если она состоит из нескольких слов, как "New York"), используются схемы тегирования:

  * **B-** (Begin): начало сущности.
  * **I-** (Inside): продолжение сущности.
  * **O** (Outside): токен не является сущностью.
  * *(Опционально)* **E-** (End) и **S-** (Single): конец и одиночная сущность.


**Структура ноутбука**

1. Часть A - "Используем готовое": делаем inference POS+NER для английского и русского, сравниваем инструменты и форматы вывода.
2. Часть B - "Best practice fine-tuning": дообучаем модель из HuggingFace 🤗 Transformers под token classification (NER/POS), обсуждаем корректную разметку BIO/BIOES и оценку качества.

По ходу будем обращать внимание на то, что в sequence labeling метки токенов связаны между собой (например, I-PER не должен появляться в никуда), и что NER обычно удобнее анализировать через найденные спаны сущностей, а не только через метки отдельных токенов.

## Часть A. Stanza: Используем готовый Pipeline

Мы используем библиотеку [Stanza](https://stanfordnlp.github.io/stanza/) от Stanford NLP Group. Это современный Python-инструмент, использующий нейросетевые модели (PyTorch) под капотом.

### Установка и загрузка моделей

In [ ]:
!pip install stanza

Загружаем модели. Stanza хранит их локально (обычно в `~/stanza_resources`). `download` достаточно выполнить один раз для каждого языка.

In [ ]:
import os
from pathlib import Path
import pandas as pd
import stanza

RES_DIR = Path("/content/stanza_resources")
RES_DIR.mkdir(parents=True, exist_ok=True)

os.environ["STANZA_RESOURCES_DIR"] = str(RES_DIR)

stanza.download("en", processors="tokenize,pos,lemma,ner", model_dir=str(RES_DIR), verbose=True)
stanza.download("ru", processors="tokenize,pos,lemma,ner", model_dir=str(RES_DIR), verbose=True)

In [ ]:
# Pipeline использует dir (не model_dir!)
nlp_en = stanza.Pipeline("en", processors="tokenize,pos,lemma,ner", dir=str(RES_DIR), use_gpu=False)
nlp_ru = stanza.Pipeline("ru", processors="tokenize,pos,lemma,ner", dir=str(RES_DIR), use_gpu=False)

doc_en = nlp_en("Barack Obama was born in Hawaii.")
doc_ru = nlp_ru("Иван Иванов приехал в Москву.")

In [4]:
print("EN ents:", [(e.text, e.type) for e in doc_en.ents])
print("RU ents:", [(e.text, e.type) for e in doc_ru.ents])

EN ents: [('Barack Obama', 'PERSON'), ('Hawaii', 'GPE')]
RU ents: [('Иван Иванов', 'PER'), ('Москву', 'LOC')]


### Построение Pipeline

В Stanza вся работа строится через объект `Pipeline`. При создании мы указываем:

1.  **Язык** (`lang`).
2.  **Список процессоров** (`processors`): какие задачи решать.
      * `tokenize`: разбиение на предложения и токены (обязательно).
      * `mwt`: Multi-Word Token expansion (актуально для фр., нем., укр. языков).
      * `pos`: части речи.
      * `lemma`: лемматизация (начальная форма слова).
      * `ner`: именованные сущности.

> Важно: Pipeline по умолчанию пытается использовать GPU. Если GPU нет, он переключится на CPU, но будет работать медленнее.

In [ ]:
# создаем Pipeline для английского языка
# ограничимся основными процессорами для скорости
nlp_en = stanza.Pipeline(
    lang="en",
    processors="tokenize,pos,lemma,ner",
    verbose=False,
    use_gpu=True # ставим False, если возникают проблемы с CUDA
)

text_en = "Steve Jobs presented the first iPhone in San Francisco in 2007."

# запуск инференса
doc_en = nlp_en(text_en)
doc_en

### Структура данных: Document, Sentence, Token, Word

Результат работы - объект `Document`. Давайте заглянем внутрь, прежде чем превращать его в таблицу.
Иерархия Stanza: `Document` -\> список `Sentence` -\> список `Token` -\> список `Word`.

> Token vs Word. В английском они часто совпадают (1 к 1). Но в некоторых языках один "Token" (подстрока) может нести несколько синтаксических "Words".
>
>   * POS и Lemma - свойства **Word**.
>   * NER - свойство **Token**.

In [7]:
# посмотрим на первое предложение
sent = doc_en.sentences[0]

print(f"Текст предложения: {sent.text}\n")

print(f"{'TOKEN':<15} {'NER (Token)':<15} {'POS (Word)':<10} {'LEMMA (Word)':<15}")
print("-" * 60)

for token in sent.tokens:
    # NER тег живет в токене
    ner_tag = token.ner

    # в английском обычно 1 токен = 1 слово
    word = token.words[0]

    print(f"{token.text:<15} {ner_tag:<15} {word.upos:<10} {word.lemma:<15}")

Текст предложения: Steve Jobs presented the first iPhone in San Francisco in 2007.

TOKEN           NER (Token)     POS (Word) LEMMA (Word)   
------------------------------------------------------------
Steve           B-PERSON        PROPN      Steve          
Jobs            E-PERSON        PROPN      Jobs           
presented       O               VERB       present        
the             O               DET        the            
first           S-ORDINAL       ADJ        first          
iPhone          S-PRODUCT       PROPN      iPhone         
in              O               ADP        in             
San             B-GPE           PROPN      San            
Francisco       E-GPE           PROPN      Francisco      
in              O               ADP        in             
2007            S-DATE          NUM        2007           
.               O               PUNCT      .              


### Анализ в Pandas


Для удобства анализа используем функцию, которая выпрямляет иерархию Stanza в плоскую таблицу.

In [12]:
def doc_to_df(doc):
    """
    Собирает информацию из уровней Token и Word в единый DataFrame.
    Важно: NER-теги берем из Token, а грамматику (POS, Lemma) из Word.
    """
    rows = []
    for sent_id, sent in enumerate(doc.sentences, start=1):
        for tok in sent.tokens:
            # NER-тег (BIOES) привязан к токену
            tok_ner = getattr(tok, "ner", "O")

            for w in tok.words:
                rows.append({
                    "sent_id": sent_id,
                    "token": tok.text,
                    "word": w.text,
                    "lemma": w.lemma,
                    "upos": w.upos, # universal POS tags
                    "xpos": w.xpos, # treebank-specific tags
                    "feats": w.feats, # морфологические признаки
                    "ner_bio": tok_ner, # био-тег токена
                    "start": tok.start_char,
                    "end": tok.end_char
                })
    return pd.DataFrame(rows)

df_en = doc_to_df(doc_en)
df_en.head(10)

,sent_id,token,word,lemma,upos,xpos,feats,ner_bio,start,end
0,1,Steve,Steve,Steve,PROPN,NNP,Number=Sing,B-PERSON,0,5
1,1,Jobs,Jobs,Jobs,PROPN,NNP,Number=Sing,E-PERSON,6,10
2,1,presented,presented,present,VERB,VBD,Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbF...,O,11,20
3,1,the,the,the,DET,DT,Definite=Def|PronType=Art,O,21,24
4,1,first,first,first,ADJ,JJ,Degree=Pos|NumForm=Word|NumType=Ord,S-ORDINAL,25,30
5,1,iPhone,iPhone,iPhone,PROPN,NNP,Number=Sing,S-PRODUCT,31,37
6,1,in,in,in,ADP,IN,None,O,38,40
7,1,San,San,San,PROPN,NNP,Number=Sing,B-GPE,41,44
8,1,Francisco,Francisco,Francisco,PROPN,NNP,Number=Sing,E-GPE,45,54
9,1,in,in,in,ADP,IN,None,O,55,57


### Работа с сущностями (Spans)

Анализировать `ner_bio` (теги B-PERSON, E-PERSON) глазами неудобно. Stanza автоматически склеивает теги в готовые объекты-сущности `doc.ents`.

In [13]:
def print_ents(doc):
    if not doc.ents:
        print("Сущности не найдены.")
        return

    print(f"{'ENTITY':<20} {'TYPE':<10} {'RANGE'}")
    print("-" * 40)
    for ent in doc.ents:
        # ent - это Span, у него есть текст, тип и позиции
        print(f"{ent.text:<20} {ent.type:<10} [{ent.start_char}, {ent.end_char}]")

print_ents(doc_en)

ENTITY               TYPE       RANGE
----------------------------------------
Steve Jobs           PERSON     [0, 10]
first                ORDINAL    [25, 30]
iPhone               PRODUCT    [31, 37]
San Francisco        GPE        [41, 54]
2007                 DATE       [58, 62]


### Русский язык (Stanza RU)

Применим тот же подход к русскому языку. Stanza использует модель, обученную на корпусе с **WikiNER**, поэтому набор тегов может отличаться.


In [ ]:
nlp_ru = stanza.Pipeline(
    "ru",
    processors="tokenize,pos,lemma,ner",
    verbose=False
)

text_ru = "Альберт Эйнштейн родился в Ульме и работал в Швейцарии. В 1921 году он получил Нобелевскую премию по физике."
doc_ru = nlp_ru(text_ru)

# выводим таблицу
df_ru = doc_to_df(doc_ru)
display(df_ru.head(15)) # display работает в ноутбуках красивее print

print("\n Найденные сущности ")
print_ents(doc_ru)

### (Advanced) Stanza Client для Stanford CoreNLP

Stanza - это не только нейронный пайплайн, но и **Python-клиент** для классического Java-библиотеки Stanford CoreNLP.
CoreNLP работает как сервер. Это полезно, если вам нужны специфические функции, которых пока нет в нейронной версии Stanza (например, Coreference Resolution или OpenIE), или вы хотите использовать старые, проверенные Java-модели.

**Требования:**

  * Установленная Java.
  * Скачанный CoreNLP (делается через Stanza).

In [16]:
import os
from stanza.server import CoreNLPClient

# cкачиваем Java-библиотеки CoreNLP в локальную папку
corenlp_dir = "./corenlp"
# выполняется один раз
if not os.path.exists(corenlp_dir):
    print("Downloading CoreNLP...")
    stanza.install_corenlp(dir=corenlp_dir)

os.environ["CORENLP_HOME"] = corenlp_dir

# запускаем клиент через Context Manager (with)
# Best Practice: сервер автоматически остановится при выходе из блока with
print("Starting CoreNLP Server...")

with CoreNLPClient(
    annotators=['tokenize', 'ssplit', 'pos', 'lemma', 'ner'],
    memory='4G',
    endpoint='http://localhost:9001', # порт 9000 часто занят в Colab
    be_quiet=True
) as client:

    text = "Albert Einstein was a German-born theoretical physicist."

    # отправляем запрос на сервер
    ann = client.annotate(text)

    # объект ann — это Protobuf объект, но структура похожа
    print("\n--- CoreNLP Result ---")
    for sent in ann.sentence:
        for token in sent.token:
            print(f"{token.word:<15} {token.pos:<5} {token.ner}")

print("Server stopped.")

Starting CoreNLP Server...

--- CoreNLP Result ---
Albert          NNP   PERSON
Einstein        NNP   PERSON
was             VBD   O
a               DT    O
German          JJ    NATIONALITY
-               HYPH  O
born            VBN   O
theoretical     JJ    TITLE
physicist       NN    TITLE
.               .     O
Server stopped.


## Часть B. Fine-tuning модели для Token Classification

В этой части семинара мы научимся дообучать (fine-tune) модель из библиотеки [🤗 Transformers](https://github.com/huggingface/transformers) для задачи классификации токенов.

### Настройка окружения

Если вы работаете в Colab, установите необходимые библиотеки. Нам также понадобится `seqeval` для подсчета метрик.


In [ ]:
import os

# Xet - механизм ускоренного скачивания больших файлов с HuggingFace Hub
# High performance mode помогает быстрее качать чекпоинты моделей
os.environ["HF_XET_HIGH_PERFORMANCE"] = "1"

# увеличиваем таймауты, чтобы скачивание не обрывалось при плохом интернете
os.environ["HF_HUB_DOWNLOAD_TIMEOUT"] = "600"
os.environ["HF_HUB_ETAG_TIMEOUT"] = "60"

In [ ]:
# transformers - сами модели
# datasets - для загрузки данных
# seqeval - метрики для NER
# accelerate - для оптимизации обучения (нужен для Trainer)

In [ ]:
!pip install -q transformers datasets seqeval evaluate accelerate huggingface_hub hf_xet

In [ ]:
import transformers
print(transformers.__version__)

4.57.2


Зададим основные параметры. Вы можете поменять `task` на "pos" или "chunk", и весь ноутбук перестроится под эту задачу.
`batch_size` можно уменьшить, если у вас мало видеопамяти.

In [ ]:
task = "ner" # можно выбрать "ner", "pos" или "chunk"
model_checkpoint = "distilbert-base-uncased" # легкая и быстрая модель (Distilled BERT)
batch_size = 16 # количество примеров, обрабатываемых за раз

### Загрузка датасета

Мы будем использовать библиотеку [🤗 Datasets](https://github.com/huggingface/datasets) для загрузки данных и метрик.
Для примера возьмем классический датасет **CoNLL-2003**.

Мы будем использовать классический бенчмарк **CoNLL-2003**.

> **Справка о датасете:**
> CoNLL-2003 состоит из новостных статей Reuters. Данные размечены вручную для трех задач.
> Формат данных использует схему **BIO (Begin-Inside-Outside)**:
>
>   * **B-XXX**: Начало сущности типа XXX.
>   * **I-XXX**: Продолжение сущности типа XXX.
>   * **O**: Токен не является именованной сущностью.
>
> **Типы сущностей:**
>
>   * `PER`: Person (Люди)
>   * `ORG`: Organization (Организации)
>   * `LOC`: Location (Локации)
>   * `MISC`: Miscellaneous (Разное - названия фильмов, событий и т.д.)

Загрузим данные с помощью библиотеки `datasets`. Она автоматически скачает файлы, распакует их и преобразует в удобный формат Arrow (эффективный по памяти).



In [ ]:
from datasets import load_dataset

datasets = load_dataset("lhoestq/conll2003")
datasets

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

Объект `datasets` - это `DatasetDict`. Он похож на словарь, где ключи - это сплиты данных:

  * `train`: для обучения.
  * `validation`: для подбора гиперпараметров и оценки в процессе обучения.
  * `test`: для финальной проверки.

Давайте посмотрим на один пример. В данных уже есть токены (текст разбит на слова) и соответствующие им метки (labels) для каждой задачи (NER, POS, Chunking).

In [ ]:
datasets["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

Метки хранятся в виде ID (целых чисел). Чтобы понять, что они значат, обратимся к атрибуту `features` нашего датасета.

In [ ]:
datasets["train"].features[f"{task}_tags"]

List(Value('int64'))

Для NER метки:

  * `0` соответствует `'O'` (Outside - не сущность).
  * Остальные 4 типа сущностей (`PER`, `ORG`, `LOC`, `MISC`) имеют префиксы:
      * `B-` (Begin) - начало сущности.
      * `I-` (Inside) - продолжение сущности.

Получим список понятных названий меток:

In [ ]:
tags_feat = datasets["train"].features[f"{task}_tags"]  # "ner_tags"

# Fallback, если имена не прописаны в метаданных (для CoNLL они есть)
if hasattr(tags_feat, "feature") and hasattr(tags_feat.feature, "names"):
    label_list = tags_feat.feature.names
else:
    label_list = ["O",
                  "B-PER", "I-PER",
                  "B-ORG", "I-ORG",
                  "B-LOC", "I-LOC",
                  "B-MISC","I-MISC"]

label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

Sanity check (Проверка целостности). Проверим, какие ID меток вообще встречаются в начале датасета, чтобы убедиться, что данные не битые.


In [ ]:
# быстрый sanity-check по части train

seen = set()
for ex in datasets["train"].select(range(2000)):
    seen.update(ex["ner_tags"])
print("seen ids:", sorted(seen), "num:", len(seen))

seen ids: [0, 1, 2, 3, 4, 5, 6, 7, 8] num: 9


Для наглядности выведем примеры с расшифрованными метками в виде таблицы.

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Нельзя выбрать больше элементов, чем есть в датасете."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])

    # проходим по колонкам и заменяем числа на названия классов
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

show_random_elements(datasets["train"])

,id,tokens,pos_tags,chunk_tags,ner_tags
0,248,"[Jul, 31, ,, Jul, 31, ,, Jul, 31, ,, Jul, 31, ,]","[16, 11, 6, 22, 11, 6, 22, 11, 6, 22, 11, 6]","[11, 12, 0, 11, 12, 0, 11, 12, 0, 11, 12, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,7493,"[The, man, entered, the, Northcote, College, swimming, pool, changing, sheds, on, Monday, and, told, the, girl, and, a, friend, :, "", You, 're, for, it, now, ., ""]","[12, 21, 38, 12, 22, 22, 21, 21, 39, 24, 15, 22, 10, 38, 12, 21, 10, 12, 21, 8, 0, 28, 41, 15, 28, 30, 7, 0]","[11, 12, 21, 11, 12, 12, 12, 12, 21, 11, 13, 11, 0, 21, 11, 12, 0, 11, 12, 0, 0, 11, 21, 13, 11, 3, 0, 0]","[0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,7640,"[Rice, options, volume, was, estimated, at, 50, contracts, ,, down, from, 56, Friday, .]","[22, 24, 21, 38, 40, 15, 11, 24, 6, 30, 15, 11, 22, 7]","[11, 12, 12, 21, 22, 13, 11, 12, 0, 3, 13, 11, 11, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,13629,"[People, were, waiting, for, the, results, to, come, out, before, buying, .]","[24, 38, 39, 15, 12, 24, 35, 37, 33, 15, 39, 7]","[11, 21, 22, 13, 11, 12, 21, 22, 15, 13, 21, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,7419,"["", Next, year, ,, the, fiscal, policy, will, have, less, margin, of, freedom, because, the, government, will, have, to, face, commitments, that, it, has, taken, on, under, different, support, schemes, for, the, banks, ,, debtors, and, firms, ,, "", the, brokerage, said, .]","[0, 22, 21, 6, 12, 16, 21, 20, 37, 17, 21, 15, 21, 15, 12, 21, 20, 37, 35, 37, 24, 15, 28, 42, 40, 15, 15, 16, 21, 24, 15, 12, 24, 6, 24, 10, 24, 6, 0, 12, 21, 38, 7]","[0, 11, 12, 0, 11, 12, 12, 21, 22, 11, 12, 13, 11, 17, 11, 12, 21, 22, 22, 22, 11, 17, 11, 21, 22, 3, 13, 11, 12, 12, 13, 11, 12, 12, 12, 12, 12, 0, 0, 11, 12, 21, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
5,13458,"[*, Boeing, Co, secures, $, 5.5, billion, in, orders, for, new, ,, larger, 747s, .]","[34, 22, 22, 42, 3, 11, 11, 15, 24, 15, 16, 6, 17, 24, 7]","[0, 11, 12, 21, 11, 12, 12, 13, 11, 13, 11, 12, 12, 12, 0]","[0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
6,2370,"[7., Calvin, Davis, (, U.S., ), 49.49]","[22, 22, 22, 4, 22, 5, 11]","[11, 12, 12, 0, 11, 0, 11]","[0, 1, 2, 0, 5, 0, 0]"
7,5890,"[Played, Saturday, :]","[22, 22, 8]","[11, 12, 0]","[0, 0, 0]"
8,10145,"[I, do, n't, know, if, I, can, go, any, faster, .]","[28, 41, 30, 37, 15, 28, 20, 37, 12, 17, 7]","[11, 21, 22, 22, 17, 11, 21, 22, 11, 12, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
9,4684,"[1., Charmaine, Crooks, (, Canada, ), two, minutes, 00.42, seconds]","[11, 22, 22, 4, 22, 5, 11, 24, 11, 24]","[11, 12, 12, 0, 11, 0, 11, 12, 11, 12]","[0, 1, 2, 0, 5, 0, 0, 0, 0, 0]"


### Препроцессинг данных

Это **самая важная часть**. Нейросети не понимают слова, они работают с числами (ID токенов). Нам нужен `Tokenizer`.


Перед подачей в модель текст нужно обработать `Tokenizer`-ом. Он превратит токены в ID из словаря модели. Мы используем `AutoTokenizer`, который сам подтянет нужный токенизатор для выбранной модели (`distilbert-base-uncased`).

> **Важно:** нам нужен именно "Fast" токенизатор (написанный на Rust), так как он поддерживает маппинг токенов на оригинальные слова, что критично для NER.

> Мы используем `DistilBERT`. Его токенизатор разбивает слова на **подслова (subwords)**.
Например: слово `embedding` может разбиться на `em`, `##bed`, `##ding`.

In [ ]:
from transformers import AutoTokenizer
import transformers

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

### Проблема разбиения слов (Subword Tokenization)

Трансформеры часто используют разбиение на подслова (subwords). Даже если входные данные уже разбиты на слова (как у нас), токенизатор может разбить одно слово на несколько частей.

Давайте посмотрим на пример:

In [ ]:
# Пример из датасета, где есть сложное слово
example = datasets["train"][4]
print("Original tokens:", example["tokens"])

tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print("BERT tokens:    ", tokens)

Original tokens: ['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.']
BERT tokens:     ['[CLS]', 'germany', "'", 's', 'representative', 'to', 'the', 'european', 'union', "'", 's', 'veterinary', 'committee', 'werner', 'z', '##wing', '##mann', 'said', 'on', 'wednesday', 'consumers', 'should', 'buy', 'sheep', '##me', '##at', 'from', 'countries', 'other', 'than', 'britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.', '[SEP]']


Видите? Слова "Zwingmann" и "sheepmeat" разбились на несколько токенов (например, `sheep`, `##me`, `##at`).

Это создает проблему: у нас была **одна метка** для слова "sheepmeat", а теперь у нас **три токена**. Как расставить метки?

К счастью, токенизатор возвращает метод `word_ids()`, который помогает сопоставить токены обратно со словами.

In [ ]:
print(tokenized_input.word_ids())

[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 11, 11, 12, 13, 14, 15, 16, 17, 18, 18, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, None]


Он возвращает список той же длины, что и токены:

  * `None` для специальных токенов (как `[CLS]`, `[SEP]`).
  * Индекс слова для обычных токенов.


### Стратегия выравнивания меток

Мы будем использовать следующую стратегию:

1.  Специальным токенам (`None`) присваиваем метку **-100**. Этот индекс игнорируется функцией потерь (loss function) в PyTorch/TensorFlow по умолчанию.
2.  Первому токену слова присваиваем оригинальную метку.
3.  Для остальных частей слова (сабтокенов) мы можем либо присвоить ту же метку, либо -100. Мы выберем вариант дублирования метки (флаг `label_all_tokens = True`).

In [ ]:
word_ids = tokenized_input.word_ids()

# пример выравнивания для одного предложения
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

39 39


Теперь напишем функцию, которая делает это для всего батча данных:


In [ ]:
def tokenize_and_align_labels(examples):
    # токенизируем батч предложений.
    # truncation=True обрежет слишком длинные предложения.
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )

    labels = []
    # проходим по каждому примеру в батче
    for i, label in enumerate(examples[f"{task}_tags"]):
        # получаем маппинг "индекс токена -> индекс слова"
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            # случай 1: Спецтокены ([CLS], [SEP]) -> ставим -100
            if word_idx is None:
                label_ids.append(-100)

            # случай 2: Это начало НОВОГО слова
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx]) # Берем метку этого слова

            # случай 3: Это продолжение того же слова (сабтокен)
            else:
                # если label_all_tokens=True, дублируем метку, иначе -100
                label_ids.append(label[word_idx] if label_all_tokens else -100)

            previous_word_idx = word_idx

        labels.append(label_ids)

    # добавляем новые метки в результат
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

Применим эту функцию ко всем датасетам сразу через `map`. Параметр `batched=True` позволяет задействовать многопоточность быстрого токенизатора.

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

### Fine-tuning модели

Теперь данные готовы. Загружаем предобученную модель `AutoModelForTokenClassification`.
Мы указываем количество меток (`num_labels`) и словари для конвертации ID \<-\> Label, чтобы вывод модели был читаемым.

In [ ]:
from transformers import AutoModelForTokenClassification

# словари для конвертации ID <-> Название метки (для удобства при инференсе)
id2label = {i: label for i, label in enumerate(label_list)}
label2id = {label: i for i, label in enumerate(label_list)}

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


> Предупреждение: Вы увидите сообщение о том, что некоторые веса (например, `vocab_layer_norm`) были отброшены, а новые (классификатор) инициализированы случайно. Это **нормально**. Мы берем "тело" модели (encoder), обученное на языке, и заменяем "голову" на новую, которую сейчас будем обучать под нашу задачу.

### Data Collator

Нам нужен Data Collator, который соберет примеры в батчи. Для классификации токенов он должен делать **padding** не только для входных ID, но и для **меток** (labels), чтобы тензоры были прямоугольными.

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)


### Метрики (seqeval)

Обычная `accuracy` плохо подходит для NER, так как класс `O` (не сущность) встречается в 90% случаев. Модель может предсказывать везде `O` и иметь высокую точность, но быть бесполезной.

Мы будем использовать библиотеку `seqeval`, которая считает Precision, Recall и F1-score на уровне **сущностей**, а не отдельных токенов.

Обычно используют evaluate, но на всякий случай добавлю fallback на load_metric.

In [ ]:
import numpy as np

try:
    import evaluate
    seqeval = evaluate.load("seqeval")
except Exception:
    from datasets import load_metric
    seqeval = load_metric("seqeval")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    true_predictions = [
        [label_list[p] for (p, l) in zip(pred, lab) if l != -100]
        for pred, lab in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(pred, lab) if l != -100]
        for pred, lab in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)

    # у seqeval бывают разные ключи в зависимости от версии, но overall_* обычно есть
    return {
        "precision": results.get("overall_precision", 0.0),
        "recall": results.get("overall_recall", 0.0),
        "f1": results.get("overall_f1", 0.0),
        "accuracy": results.get("overall_accuracy", 0.0),
    }

### Запуск обучения

Мы готовы обучать\! Добавим `metric_callback`. Обучение (Fine-tuning) на PyTorch через Trainer.


In [ ]:
from transformers import TrainingArguments, Trainer
import torch

num_train_epochs = 3 # количество проходов по датасету

args = TrainingArguments(
    output_dir=f"./{task}_distilbert_finetuned", # куда сохранять модель
    learning_rate=2e-5, # скорость обучения (обычно 2e-5 ... 5e-5)
    weight_decay=0.01, # регуляризация
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,

    eval_strategy="epoch", # проверять качество в конце каждой эпохи
    save_strategy="epoch", # сохранять чекпоинт в конце каждой эпохи
    logging_strategy="steps",
    logging_steps=50, # логировать лосс каждые 50 шагов
    report_to="none", # отключаем wandb для простоты
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    processing_class=tokenizer, # (или tokenizer=tokenizer в старых версиях)
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# запуск обучения
trainer.train()
trainer.evaluate()

/tmp/ipython-input-4124859767.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.083600,0.073935,0.894765,0.919790,0.907105,0.979030
2,0.037200,0.059971,0.924810,0.937018,0.930874,0.983415
3,0.027400,0.060505,0.928848,0.939031,0.933912,0.984479


{'eval_loss': 0.0605047307908535,
 'eval_precision': 0.9288480690494633,
 'eval_recall': 0.9390312115449155,
 'eval_f1': 0.9339118825100133,
 'eval_accuracy': 0.9844790061480293,
 'eval_runtime': 2.6793,
 'eval_samples_per_second': 1213.021,
 'eval_steps_per_second': 76.14,
 'epoch': 3.0}

### Инференс (Проверка)

После обучения проверим модель. Мы можем загрузить её через Pipeline API, который скроет от нас всю сложность препроцессинга.

In [ ]:
# Быстро через pipeline (PyTorch)

from transformers import pipeline

token_classifier = pipeline(
    "token-classification",
    model=trainer.model,
    tokenizer=tokenizer,
    aggregation_strategy="simple", # Важно: склеивает B-PER и I-PER в одну сущность
    framework="pt",
)

sample_sentence = "Steve Jobs presented the first iPhone in San Francisco in 2007."

results = token_classifier(sample_sentence)
print("Input:", sample_sentence)
print("\nPredictions:")
for res in results:
    print(f"{res['entity_group']}: {res['word']} (score: {res['score']:.2f})")

Device set to use cuda:0


Input: Steve Jobs presented the first iPhone in San Francisco in 2007.

Predictions:
PER: steve jobs (score: 0.97)
MISC: iphone (score: 0.91)
LOC: san francisco (score: 0.99)


In [ ]:
# ручной инференс (без pipeline)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval() # переводим в режим оценки (выкл dropout и т.д.)

text = "Steve Jobs presented the first iPhone in San Francisco in 2007."
# токенизируем и превращаем в тензоры PyTorch
inputs = tokenizer(text, return_tensors="pt").to(device)

with torch.no_grad():
    logits = model(**inputs).logits  # размерность: [1, seq_len, num_labels]

# получаем ID класса с максимальной вероятностью
pred_ids = logits.argmax(dim=-1)[0].cpu().tolist()
input_ids = inputs["input_ids"][0].cpu().tolist()

# декодируем токены обратно в текст
tokens = tokenizer.convert_ids_to_tokens(input_ids)
# декодируем ID классов в названия (PER, ORG...)
labels = [model.config.id2label[i] for i in pred_ids]

print("\nПотокенный вывод:")
for t, y in zip(tokens, labels):
    print(f"{t:>12} -> {y}")

       [CLS] -> O
       steve -> B-PER
        jobs -> I-PER
   presented -> O
         the -> O
       first -> O
      iphone -> B-MISC
          in -> O
         san -> B-LOC
   francisco -> I-LOC
          in -> O
        2007 -> O
           . -> O
       [SEP] -> O


Результат pipeline автоматически сгруппировал токены и выдал нам список найденных сущностей с их типами и уверенностью (score).

### Поменяем task?

In [ ]:
task = "pos"
label_col = f"{task}_tags"   # "pos_tags"

tags_feat = datasets["train"].features[label_col]

if hasattr(tags_feat, "feature") and hasattr(tags_feat.feature, "names"):
    label_list = tags_feat.feature.names
else:
    # просто делаем имена L0..L{K-1} по числу классов, найденному в данных
    all_ids = set()
    for split in ["train", "validation", "test"]:
        for ex in datasets[split]:
            all_ids.update(ex[label_col])
    num_labels = max(all_ids) + 1
    label_list = [f"L{i}" for i in range(num_labels)]

id2label = {i: l for i, l in enumerate(label_list)}
label2id = {l: i for i, l in enumerate(label_list)}

In [ ]:
import numpy as np
import evaluate
from sklearn.metrics import f1_score

acc = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    y_true, y_pred = [], []
    for p_seq, l_seq in zip(preds, labels):
        for p, l in zip(p_seq, l_seq):
            if l != -100:
                y_true.append(int(l))
                y_pred.append(int(p))

    return {
        "accuracy": acc.compute(predictions=y_pred, references=y_true)["accuracy"],
        "macro_f1": f1_score(y_true, y_pred, average="macro"),
    }

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id,
)

In [ ]:
ta_params = inspect.signature(TrainingArguments.__init__).parameters

ta_kwargs = dict(
    output_dir="./pos_distilbert_pt",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,
    report_to="none",
)

if "eval_strategy" in ta_params:
    ta_kwargs["eval_strategy"] = "epoch"
else:
    ta_kwargs["evaluation_strategy"] = "epoch"

args = TrainingArguments(**ta_kwargs)

In [ ]:
trainer_kwargs = dict(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer_params = inspect.signature(Trainer.__init__).parameters
if "processing_class" in trainer_params:
    trainer_kwargs["processing_class"] = tokenizer
else:
    trainer_kwargs["tokenizer"] = tokenizer

trainer = Trainer(**trainer_kwargs)
trainer.train()
trainer.evaluate()

In [ ]:
pos_tagger = pipeline(
    "token-classification",
    model=trainer.model,
    tokenizer=tokenizer,
    aggregation_strategy="simple",
    framework="pt",
)

sentence = "Steve Jobs presented the first iPhone in San Francisco in 2007."
out = pos_tagger(sentence)

for r in out[:25]:
    print(r["word"], "->", r["entity_group"], f"({r['score']:.2f})")